# Making Stuff Interactive: **ipywidgets**

[🔗 Read the docs](https://ipywidgets.readthedocs.io/en/stable/)

You can have a look at all the different widgets [here](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html).

Enabling interaction with matplotlib charts in the Jupyter notebook and JupyterLab

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

import ipywidgets as widgets

import warnings
warnings.filterwarnings("ignore",category=mpl.cbook.mplDeprecation)

# Creating a Simple Counter Button

In [ ]:
button = widgets.Button(description='0')
button

In [ ]:
def increase_counter(b):
    b.description = str(int(b.description) + 1)

button.on_click(increase_counter)

# Adding a Slider to a Plot

In [ ]:
plt.ioff()

slider = widgets.FloatSlider(
    orientation='horizontal',
    description='Frequency:',
    value=1.0,
    min=0.1,
    max=10.0)
slider.layout.width = '750px'

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.layout.min_height = '400px'
ax.set_title(f'Plotting: {slider.value} Hz Sine')
ax.set_xlabel('t in s')
ax.set_ylabel('Amplitude')
ax.grid(True)

x = np.linspace(0, 1, 500)
lines = ax.plot(x, np.sin(slider.value * x * 2. * np.pi))

def update_lines(change):
    plt.title(f'Plotting: {change.new} Hz Sine')
    lines[0].set_data(x, np.sin(change.new * x * 2. * np.pi))
    fig.canvas.draw()
    fig.canvas.flush_events()

slider.observe(update_lines, names='value')

widgets.AppLayout(
    center=fig.canvas,
    footer=slider,
    pane_heights=[0, 6, 1])

# Librosa example

In [ ]:
import librosa
import librosa.display
from IPython.display import Audio

Librosa includes some example audio material that you can retrieve by using the ```ex``` method.

The first time you retrieve a particular example, it will be downloaded from the remote repository and cashed locally for further use.

In [ ]:
data, fs = librosa.load(librosa.ex('trumpet'))

In [ ]:
Audio(data=data, rate=fs)

Using ```ipywidgets```, we can build a small toy app to explore the the audio file further.

In [ ]:
S = librosa.power_to_db(librosa.feature.melspectrogram(y=data, sr=fs), ref=np.max)
fs_mel = librosa.mel_frequencies(fmax=8192)
n_frames = S.shape[1]
frame_idx = 0

In [ ]:
plt.ioff()
fig, axes = plt.subplots(nrows=2, figsize=(8, 6))
fig.canvas.header_visible = False

# plot magnitude spectrum of a single stft frame in upper subplot
lines = axes[0].plot(fs_mel, S[:, frame_idx], '.-', color='#ff5722')
axes[0].set_title(f'Frame #{frame_idx}')
axes[0].set_xlabel('f in Hz')
axes[0].set_ylabel('dB')
axes[0].set_ylim([-85, 5])
axes[0].set_xscale('symlog', linthresh=1000, base=2)
axes[0].set_xticks([2**x for x in range(9, 14)])
axes[0].set_xticklabels([str(x) for x in axes[0].get_xticks()])
axes[0].grid(b=True, which='both', alpha=0.4)

# plot the whole spectrogram in lower subplot 
librosa.display.specshow(S, x_axis='s', y_axis='mel', sr=fs, ax=axes[1], cmap=mpl.cm.viridis)

# plot vertical line over spectrogram to highlight current frame 
vlines = axes[1].axvline(x=librosa.frames_to_time(frame_idx + 0.5), color=lines[0].get_color(), linewidth=1)

# set up the slider widget
slider = widgets.IntSlider(value=frame_idx, min=0, max=S.shape[1] - 1, description='Frame')
slider.layout.width = '800px'

def update(change):
    axes[0].set_title(f'Frame #{change.new}')
    lines[0].set_data(fs_mel, S[:, change.new])
    vlines.set_xdata([librosa.frames_to_time(change.new + 0.5) for _ in range(2)])
    fig.canvas.draw()
    fig.canvas.flush_events()
    
slider.observe(update, names='value')

widgets.AppLayout(
    center=fig.canvas,
    footer=slider,
    pane_heights=[0, 6, 1])

# Using `interact`

At the most basic level, `interact` autogenerates UI controls for function arguments, and then calls the function with those arguments when you manipulate the controls interactively.

In [ ]:
def f(x):
    return x

When you pass this function as the first argument to `interact` along with an integer keyword argument (`x=10`), a slider is generated and bound to the function parameter.

In [ ]:
widgets.interact(f, x = 10);

When you move the slider, the function is called, and its return value is printed.

You can also pass different types as the keyword argument `x`:

In [ ]:
widgets.interact(f, x = True);

In [ ]:
widgets.interact(f, x = 'hello');